# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Examples on storage solutions for Big Data (PostgreSQL)** </center>
---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [2]:
import findspark
findspark.init()

#### Create SparkSession 


In [3]:
from pyspark.sql import SparkSession

# Hint for homework: .config("spark.jars.packages", "groupId:artifactId:version") \

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions-Postgres-Write") \
    .master("spark://eeebc5f4a9c2:7077") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.13:3.4.1") \
    .config("spark.cassandra.connection.host", "my-cassandra") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bf899567-c6bd-44a2-80ec-75eda90fd8ef;1.0
	confs: [default]


KeyboardInterrupt: 

#### Load Netflix Dataset

In [4]:
from team_name.spark_utils import SparkUtils

columns_info = [("show_id", "string"),("type", "string"),("title", "string"),("director", "string"),("country", "string"),("date_added", "string"),("release_year", "integer"),("rating", "string"), ("duration", "string"),("listed_in", "string")]

netflix_schema = SparkUtils.generate_schema(columns_info)

netflix_df = spark.read \
                .schema(netflix_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/netflix1.csv")

netflix_df.printSchema()

netflix_df.show(5, truncate=False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)



+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead            |Kirsten Johnson|United States|9/25/2021 |2020        |PG-13 |90 min  |Documentaries                                                |
|s3     |TV Show|Ganglands                       |Julien Leclercq|France       |9/24/2021 |2021        |TV-MA |1 Season|Crime TV Shows, International TV Shows, TV Action & Adventure|
|s6     |TV Show|Midnight Mass                   |Mike Flanagan  |United States|9/24/

#### Write DF in Postgres

In [5]:
jdbc_url = "jdbc:postgresql://postgres-iteso:5432/postgres"

netflix_df.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "netflix") \
    .option("user", "postgres") \
    .option("password", "Admin@1234") \
    .option("driver", "org.postgresql.Driver") \
    .save()

print("Done")

Done


In [ ]:
# Escribir el DataFrame a Cassandra
netflix_df.write \
    .format("org.apache.spark.sql.cassandra") \
    .mode("overwrite") \
    .options(table="netflix_table", keyspace="spark_cassandra") \
    .save()

print("Done!")

In [ ]:
# Leer datos desde Cassandra
df_from_cassandra = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="netflix_table", keyspace="spark_cassandra") \
    .load()

df_from_cassandra.show()

+-------+-------------+----------+--------------------+---------+--------------------+------+------------+--------------------+-------+
|show_id|      country|date_added|            director| duration|           listed_in|rating|release_year|               title|   type|
+-------+-------------+----------+--------------------+---------+--------------------+------+------------+--------------------+-------+
|  s3700|        India| 6/30/2019|       Robbie Grewal|  141 min|Dramas, Internati...| TV-14|        2019|  Romeo Akbar Walter|  Movie|
|  s3300|        India| 11/7/2019|         Pawan Kumar|  121 min|International Mov...| TV-14|        2016|              U Turn|  Movie|
|  s6485|     Pakistan| 1/15/2019|          Mohsin Ali|  118 min|Comedies, Interna...| TV-14|        2017|       Chupan Chupai|  Movie|
|  s6200|     Bulgaria|  9/8/2017|         Gabe Ibáñez|  110 min|International Mov...|     R|        2014|            Automata|  Movie|
|  s6329|    Argentina|  7/1/2017|       Martin 

In [1]:
# Stop the SparkContext
sc.stop()

NameError: name 'sc' is not defined

#### Create a DF from PostgreSQL

In [12]:
spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions-Postgres-Read") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.jars", "/home/jovyan/notebooks/jars/postgresql-42.7.5.jar") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

##### Set Database connection properties 

In [ ]:
# Database connection properties
jdbc_url = "jdbc:postgresql://postgres-iteso:5432/postgres"
db_properties = {
      "user": "postgres",
      "password": "Admin@1234",
      "driver": "org.postgresql.Driver"
  }

df = spark.read \
    .jdbc(url=jdbc_url, table="netflix", properties=db_properties)

df.printSchema()
df.show(5, False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)

+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead           

In [2]:
sc.stop()

NameError: name 'sc' is not defined